In [8]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json
import pandas as pd
import datetime as dt
import plotly

In [9]:
# select json to generate data from
data = "/home/bean/code/hrf/prod_evironments/deploy/src/generate_visuals/southkorea_combined.json"

In [10]:
def sec_to_datetime(sec):
    import datetime

    convert = str(datetime.timedelta(seconds= sec))
    return convert

def find_color_from_colname(name):

    color = color_map[name]
    print(color)
    return color

def create_df_all_models(data):
    '''creates formatted df from combined model data, and returns two dfs, one reversed'''
    with open(data) as f:
        data = json.load(f)
    


    #f = open("demofile.txt", "r")
    # generate variables to be added to the df
    vidlength = data['videoInfo']['videoLength']
    unique_keys = set([_key for key,val in data['seconds'].items() for _key, _val in val.items()])
    df = pd.DataFrame({'seconds': range(0, vidlength +1)})
    #df['seconds'] = pd.to_datetime(df["seconds"], unit='s').dt.strftime('%H:%M:%S')

    #add placeholder values
    for unique_key in unique_keys:
        df[unique_key] = 0
    
    #impute real values in their corresponding seconds
    
    for second, vals in data['seconds'].items():
        for _key, _val in vals.items():

            df.at[int(second), _key] = _val
    #df['seconds'] = pd.to_datetime(df['seconds'])
    
    # rename columns and clean names
    df.columns = map(str.title, df.columns)
    df.columns = df.columns.str.replace("_", " ")
    df = df.rename(columns={"Officer": "Uniformed Person", "Civilian": "Non-Uniformed Person"})
    # reorder columns
    #df = df[['seconds', 'Officer', 'Civilian', 'Chemical Smoke', 'Riot Shield', 'Baton', '']]
    print(df.columns)

    #add negative values for graph flip
    df2 = df * -1
    df2['Seconds'] = df['Seconds'] * 1
    df = df.append(df2)

    #convert seconds to datetime
    df['Seconds'] = df['Seconds'].apply(sec_to_datetime)

    # make index seconds
    #df = df.set_index('Seconds')
    # flip df to columns being seconds, clean and rename as needed
    #df = df.T
    #df = df.reset_index()
    #df = df.rename(columns={'index': 'class'})

    # add color to each class row


    #df['color'] = ''
    #df['color'] = df['class'].map(lambda x: color_map[x])

    return df

df = create_df_all_models(data)



Index(['Seconds', 'Spray', 'Chemical Smoke', 'Riot Shield', 'Non Uniformed',
       'Baton', 'Throwing', 'Crowd', 'Running', 'Uniformed',
       'Person On Ground'],
      dtype='object')


In [11]:
df['Seconds'] = pd.to_datetime(df['Seconds'])

In [12]:
df['Seconds'].dtypes

dtype('<M8[ns]')

In [13]:
import altair as alt

interval = alt.selection_interval(encodings=['x'])

base = alt.Chart(df).transform_fold(
    list(df.columns[1:-1]), as_=['column', 'value']
).properties(    
    width=1920,
    height=200,
)


chart = base.mark_bar().encode(
    alt.X('column:N', scale=alt.Scale(domain=interval.ref()), bin=alt.Bin(maxbins=150, extent=interval), axis=alt.Axis(labels=False, title=None, ticks=False), ),
    alt.Y('value:Q'),
)
chart

alt.Chart(...)

In [342]:
df['Seconds'][0].dtypes

dtype('<M8[ns]')

In [18]:
df

,Seconds,Spray,Chemical Smoke,Riot Shield,Non Uniformed,Baton,Throwing,Crowd,Running,Uniformed,Person On Ground
0,2022-10-25 00:00:00,0,0,0,0,0,0,0,0,0,0
1,2022-10-25 00:00:01,0,1,0,0,0,0,1,0,0,0
2,2022-10-25 00:00:02,0,1,0,0,0,0,1,0,0,0
3,2022-10-25 00:00:03,0,1,0,0,0,0,1,0,0,0
4,2022-10-25 00:00:04,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
206,2022-10-25 00:03:26,0,0,-1,0,0,0,0,0,0,0
207,2022-10-25 00:03:27,0,0,0,0,0,0,0,0,0,0
208,2022-10-25 00:03:28,0,0,0,-6,0,0,0,0,0,0
209,2022-10-25 00:03:29,0,0,0,-7,0,0,0,0,0,0


In [17]:
import altair as alt

interval = alt.selection_interval(encodings=['x'])

base = alt.Chart(df).properties(    
    width=1920,
    height=50,
)

chart = base.mark_bar().encode(
    alt.X('hoursminutesseconds(Seconds):O', bin=alt.Bin(maxbins=1000, extent=interval), axis=alt.Axis(labels=False, title=None, ticks=False), scale=alt.Scale(domain=interval.ref())),
    alt.Y(alt.repeat('row'), type='quantitative', axis=alt.Axis(titleAngle=0, titlePadding=80, labels=False, ticks=False)),
    #color=alt.Color(
        #alt.repeat('column'), type='ordinal',
        #scale=alt.Scale(domain= [key for key, val in color_map.items() for col in df.columns if col == key], range=[val for key, val in color_map.items() for col in df.columns if col == key]),
    #)

).properties(
    width=1920
).repeat(
    row=list(df.columns)[1:],
    spacing=0
)
#chart.resolve_scale(color='independent')

view = base.mark_bar().encode(
    alt.X('hoursminutesseconds(Seconds):O', title='click and drag and use mousewheel above to zoom', axis=alt.Axis()),
).add_selection(
    interval
).properties(
    height=100,
    width=1920
)

both = alt.VConcatChart(
    vconcat=[view, chart],
    spacing=100
).configure_axis(
    #gridDash=[100,1]
    #grid=True,
)
chart

alt.RepeatChart(...)

In [170]:
df

Seconds,class,0:00:00,0:00:01,0:00:02,0:00:03,0:00:04,0:00:05,0:00:06,0:00:07,0:00:08,...,0:03:22,0:03:23,0:03:24,0:03:25,0:03:26,0:03:27,0:03:28,0:03:29,0:03:30,color
0,Non Uniformed,0,0,0,0,0,1,0,0,0,...,-1,-2,-3,0,0,0,-6,-7,-7,purple
1,Riot Shield,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,-1,0,0,0,0,green
2,Throwing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,red
3,Spray,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,red
4,Uniformed,0,0,0,0,0,0,0,0,0,...,0,0,0,-1,0,0,0,0,0,purple
5,Baton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,green
6,Chemical Smoke,0,1,1,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,green
7,Running,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,yellow
8,Person On Ground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,yellow
9,Crowd,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,yellow


In [19]:
import altair as alt

interval = alt.selection_interval(encodings=['x'])

base = alt.Chart(df).properties(    
    width=1920,
    height=50,
)

chart = base.mark_bar().encode(
    alt.X('Seconds', bin=alt.Bin(maxbins=150, extent=interval), axis=alt.Axis(labels=False, title=None, ticks=False), scale=alt.Scale(domain=interval.ref())),
    alt.Y(alt.repeat('row'), type='quantitative', axis=alt.Axis(titleAngle=0, titlePadding=80, labels=False, ticks=False)),
    #color=alt.Color(
        #alt.repeat('column'), type='ordinal',
        #scale=alt.Scale(domain= [key for key, val in color_map.items() for col in df.columns if col == key], range=[val for key, val in color_map.items() for col in df.columns if col == key]),
    #)

).properties(
    width='container'
).repeat(
    row=list(df.columns)[1:],
    spacing=0
)

#chart.resolve_scale(color='independent')

view = base.mark_bar().encode(
    alt.X('Seconds', title='click and drag and use mousewheel above to zoom', ),
).add_selection(
    interval
).properties(
    height=100,
    width=1920
)

both = alt.VConcatChart(
    vconcat=[view, chart],
    spacing=100
).configure_axis(
    gridDash=[100,1]
    #grid=True,
)

(both).save('chart.json')
#view & chart
both

alt.VConcatChart(...)

Seconds,class,0:00:00,0:00:00,0:00:01,0:00:01,0:00:02,0:00:02,0:00:03,0:00:03,0:00:04,...,0:03:26,0:03:26,0:03:27,0:03:27,0:03:28,0:03:28,0:03:29,0:03:29,0:03:30,0:03:30
0,Non Uniformed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,6,-6,7,-7,7,-7
1,Riot Shield,0,0,0,0,0,0,0,0,0,...,1,-1,0,0,0,0,0,0,0,0
2,Throwing,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Spray,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Uniformed,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Baton,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Chemical Smoke,0,0,1,-1,1,-1,1,-1,1,...,0,0,0,0,0,0,0,0,0,0
7,Running,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Person On Ground,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Crowd,0,0,1,-1,1,-1,1,-1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
with open('chart.json') as f:
    data = json.load(f)

In [12]:

list(data['datasets'].keys())[0]
import altair_saver
(view & chart).altair_saver.save(f"temp_videodata_storage/chartviz.png")

AttributeError: 'VConcatChart' object has no attribute 'altair_saver'

In [ ]:
# matplotlib
%matplotlib ipympl
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.widgets import Slider
from mpl_interactions import interactive_hist
import numpy as np

df_col_names = list(df.columns[1:])

fig, axs = plt.subplots(len(df.columns)- 1, 1, figsize=(25,10))

def draw_graph():
    for ind, ax in enumerate(axs):
        #color
        col_name = df.columns[ind + 1]


        ax.hist(x=df['Seconds'],bins=200, weights=df[df_col_names[ind]], color=color_map[col_name])
        ax.hist(x=df['Seconds'],bins=200, weights=df[df_col_names[ind]] * -1, color=color_map[col_name])
        ax.set_ylabel(df_col_names[ind], rotation=0, ha='right')

        ax.margins(x=0)
        ax.axes.yaxis.set_ticklabels([])
        ax.axes.yaxis.set_ticks([])
        if ind == 0:
            ax.xaxis.tick_top()
        else:
            ax.axes.xaxis.set_ticklabels([])
            
        #controls = interactive_hist(arr=df['Seconds'],bins=200, weights=df[df_col_names[ind]], color=color_map[col_name], scale=(0, 1))
        controls = interactive_hist(arr=df['Seconds'].to_numpy(), scale=(0, 1))
        
plt.subplots_adjust(wspace=0, hspace=0)

draw_graph()

# SLIDER




"""
ax_slide=plt.axes([.25,.1,.65,.03])
s_factor=Slider(ax_slide,"changing value",valmin=0,valmax=1.0,valinit=.5,valstep=0.01)

def update(val):
    pass
 
s_factor.on_changed(update)
"""
plt.show()


In [99]:
df

,Seconds,Spray,Throwing,Person On Ground,Non-Uniformed Person,Uniformed Person,Chemical Smoke,Baton,Riot Shield,Crowd,Brawling,Running
0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,9,4,0,0,0,0,0,0
2,2,0,1,0,2,0,0,0,0,0,0,0
3,3,0,1,0,7,0,0,0,0,0,0,0
4,4,0,1,0,6,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
136,136,0,0,0,0,5,0,0,0,0,0,0
137,137,0,0,0,3,5,0,0,0,0,0,0
138,138,0,0,0,6,4,0,0,1,0,0,0
139,139,0,0,0,4,5,0,0,0,0,0,0


In [53]:
# fig 2 with histogram
#keep track of detection colors
bins = 200

#detection_colors = {'Officer': 'blue', 'Civilian': 'blue', 'Baton': 'green', 'Gun':'green', 'Riot Shield': 'green', 'Chemical Smoke': 'red', 'Pepper Spray': 'red'}

x = df['seconds']
y = df['Officer']

fig = make_subplots(
    rows=len(df.columns)-1, 
    cols=1,
    shared_xaxes=True,
    subplot_titles=df.columns[1:],
    vertical_spacing=0,
    column_widths=[1],
    
    )

# add the bars into the graph
for i, val in enumerate(df.columns):
    if val == 'seconds':
        continue
    fig.add_trace(
        go.Histogram(
        x=df['seconds'],
        y=df[val],
        histfunc='sum',
        marker=dict(color='blue'),
        #marker=dict(color=detection_colors[val]),
        nbinsx=bins
    ),row=i, col=1),
    # add negative of same values to flip the bars on the same graph
    fig.add_trace(
        go.Histogram(
        x=df['seconds'],
        y=df[val] * -1,
        histfunc='sum',
        nbinsx=bins,
        marker=dict(color='blue')
        #marker=dict(color=detection_colors[val]),
        
    ),row=i, col=1)

fig.update_layout(
    showlegend=False,
    height = 400,
    width = 1500,
    barmode='relative',
    bargap=0,
    bargroupgap=0,
    margin=dict(
    l=150,
    r=10,
    b=20,
    t=30,
    pad=0
    ),
    )

for i in range(1, len(df.columns)):
    fig.update_yaxes(
    row=i, col=1, side='left', showticklabels=False, showgrid=False, linewidth=1, linecolor='black', mirror=True,)

    if i == 1:
        fig.update_xaxes(row=i, col=1, side='top',linewidth=1, linecolor='gray', mirror=True,showticklabels=True, nticks=12)
    else:
        fig.update_xaxes(row=i, col=1, showticklabels=False, linewidth=1, linecolor='gray', mirror=True)

for i in range(0, len(df.columns[1:])):
    fig.layout.annotations[i].update(x=-0.06)
    fig.layout.annotations[i]['y'] = fig.layout.annotations[i]['y'] - 0.08

fig.show()

In [104]:
import json
import pandas as pd
import datetime as dt
import altair as alt

def generate_visuals(path):

    

    def format_and_export_plotly_to_json(chart):
        export_json_name = f'/home/bean/code/hrf/prod_evironments/deploy/src/generate_visuals/southkorea_combined.json'
        (chart).save(export_json_name)

        with open(export_json_name) as f:
            data = json.load(f)
        
        # make edits to final visual json
        # append time list to end of data list
        data['duration'] = data['datasets'][list(data['datasets'].keys())[0]][-1]['Seconds']
        
        with open(export_json_name, "w") as outfile:
            json.dump(data, outfile)
        
        """
        doesn't work with altair_saver. needs conda-forge install, which was upgrading pytorch
        if dev_mode == True:
            (view & chart).save(f"temp_videodata_storage/{video_id}_chartviz.png")"""


    def create_df_all_models():
        '''creates formatted df from combined model data, and returns two dfs, one reversed'''

        with open(f"{path}") as f:
            data = json.load(f)
        
        correct_class_order = ['Seconds', 'Uniformed Person', 'Non-Uniformed Person', 'Brawling', 'Crowd','Person On Ground', 'Restraining', 
                                'Running', 'Spray', 'Striking', 'Throwing', 'Baton', 'Chemical Smoke', 'Gun', 'Riot Shield']

        # generate variables to be added to the df
        vidlength = data['videoInfo']['videoLength']
        unique_keys = set([_key for key,val in data['seconds'].items() for _key, _val in val.items()])
        df = pd.DataFrame({'seconds': range(0, vidlength +1)})
        #df['seconds'] = pd.to_datetime(df["seconds"], unit='s').dt.strftime('%H:%M:%S')

        #add placeholder values
        for unique_key in unique_keys:
            df[unique_key] = 0
        
        #impute real values in their corresponding seconds
        for second, vals in data['seconds'].items():
            for _key, _val in vals.items():
                df.at[int(second), _key] = _val

        #df['seconds'] = pd.to_datetime(df['seconds'])
        
        # rename columns and clean names
        df.columns = map(str.title, df.columns)
        df.columns = df.columns.str.replace("_", " ")

        #TODO: THIS CODE IS WEAKSAUCE
        try:
            df = df.rename(columns={"Non Uniformed": "Non-Uniformed Person"})
            df = df.rename(columns={"Uniformed": "Uniformed Person"})
        except:
            pass

        df2 = df * -1
        df2['Seconds'] = df['Seconds'] * 1
        df = df.append(df2)
        df.sort_index()

        # reorder df to correct class order
        cols = list(df.columns)

        for val in correct_class_order:
            if val not in cols:
                correct_class_order.remove(val)

        actual_col_order = correct_class_order
        df = df[actual_col_order]

        return df


    df = create_df_all_models()
    
    # SIMULATION CODE
    #df = df[0:120]

    # generate graph

    # calculate bin
    vid_length_sec = len(df) / 2
    if vid_length_sec <= 120:
        bin_size = vid_length_sec * 3
    else:
        bin_size = 240
    
    # calculate amount of ticks to use


    interval = alt.selection_interval(encodings=['x'], init={'x':[0, (len(df) / 2)]})

    base = alt.Chart(df).properties(    
        width=1920,
        height=50,
    )

    chart = base.mark_bar().encode(
        alt.X('Seconds', bin=alt.Bin(maxbins=bin_size, extent=interval), axis=alt.Axis(labels=False, title=None, ticks=False), scale=alt.Scale(domain=interval.ref())),
        alt.Y(alt.repeat('row'), type='quantitative', axis=alt.Axis(titleAngle=0, titlePadding=80, labels=False, ticks=False)),
        
        #color=alt.Color(
            #alt.repeat('column'), type='ordinal',
            #scale=alt.Scale(domain= [key for key, val in color_map.items() for col in df.columns if col == key], range=[val for key, val in color_map.items() for col in df.columns if col == key]),
        #)
    ).properties(
        #width=1920
    ).repeat(
        row=list(df.columns)[1:],
        spacing=0,
        
    )

    #chart.resolve_scale(color='independent')

    view = base.mark_bar().encode(
        alt.X('Seconds:Q', title='Use mousewheel and drag above to zoom', axis=alt.Axis(tickCount=10)),
    ).add_selection(
        interval
    )

    both = alt.VConcatChart(
        vconcat=[view, chart],
        spacing=100,
        padding={"left": 10, "top": 10, "right": 10, "bottom": 10},
    ).configure_axis(
        gridDash=[100,1]
        #grid=True,
    )

    #(view & chart).save('chart.json')
    #view & chart
    return both
    #return format_and_export_plotly_to_json(both)
both = generate_visuals(path='/home/bean/code/hrf/prod_evironments/deploy/src/generate_visuals/southkorea_combined.json')

both

     Seconds  Uniformed Person  Non-Uniformed Person  Crowd  Person On Ground  \
0          0                 0                     0      0                 0   
1          1                 0                     0      1                 0   
2          2                 0                     0      1                 0   
3          3                 0                     0      1                 0   
4          4                 0                     0      0                 0   
..       ...               ...                   ...    ...               ...   
206      206                 0                     0      0                 0   
207      207                 0                     0      0                 0   
208      208                 0                    -6      0                 0   
209      209                 0                    -7      0                 0   
210      210                 0                    -7      0                 0   

     Running  Spray  Throwi

alt.VConcatChart(...)

In [193]:
list(df['Seconds'][-1:])

[Timestamp('2022-10-25 00:03:30')]

In [203]:
list(df['Seconds'][0:1])

[Timestamp('2022-10-25 00:00:00')]

# GOD HELP US ALL

In [386]:
import json
from time import time
import pandas as pd
import datetime as dt
import altair as alt

def generate_visuals(path):

    def sec_to_datetime(sec):
            import datetime

            convert = str(datetime.timedelta(seconds= sec))
            return convert  

    def format_and_export_plotly_to_json(chart):
        export_json_name = f'/home/bean/code/hrf/prod_evironments/deploy/src/generate_visuals/southkorea_combined.json'
        (chart).save(export_json_name)

        with open(export_json_name) as f:
            data = json.load(f)
        
        # make edits to final visual json
        # append time list to end of data list
        data['duration'] = data['datasets'][list(data['datasets'].keys())[0]][-1]['Seconds']
        
        with open(export_json_name, "w") as outfile:
            json.dump(data, outfile)
        
        """
        doesn't work with altair_saver. needs conda-forge install, which was upgrading pytorch
        if dev_mode == True:
            (view & chart).save(f"temp_videodata_storage/{video_id}_chartviz.png")"""


    def create_df_all_models():
        '''creates formatted df from combined model data, and returns two dfs, one reversed'''

        with open(f"{path}") as f:
            data = json.load(f)
        
        correct_class_order = ['SECONDS', 'UNIFORMED PERSON', 'NON-UNIFORMED PERSON', 'CROWD','PERSON ON GROUND', 'RESTRAINING', 
                                'RUNNING', 'BRAWLING', 'SPRAY', 'STRIKING', 'THROWING', 'GUN', 'BATON', 'CHEMICAL SMOKE', 'RIOT SHIELD', 'PEPPER SPRAY']

        # generate variables to be added to the df
        vidlength = data['videoInfo']['videoLength']
        unique_keys = set([_key for key,val in data['seconds'].items() for _key, _val in val.items()])
        df = pd.DataFrame({'seconds': range(0, vidlength +1)})
        #df['seconds'] = pd.to_datetime(df["seconds"], unit='s').dt.strftime('%H:%M:%S')

        #add placeholder values
        for unique_key in unique_keys:
            df[unique_key] = 0
        
        #impute real values in their corresponding seconds
        for second, vals in data['seconds'].items():
            for _key, _val in vals.items():
                df.at[int(second), _key] = _val

        #df['seconds'] = pd.to_datetime(df['seconds'])
        
        # rename columns and clean names
        df.columns = map(str.title, df.columns)
        df.columns = df.columns.str.replace("_", " ")

        #TODO: THIS CODE IS WEAKSAUCE
        try:
            df = df.rename(columns={"Non Uniformed": "Non-Uniformed Person"})
        except:
            pass
        try:
            df = df.rename(columns={"Uniformed": "Uniformed Person"})
        except:
            pass

        df2 = df * -1
        df2['Seconds'] = df['Seconds'] * 1
        df = df.append(df2)
        df.sort_index()
        df.columns = df.columns.str.upper()
        # reorder df to correct class order

        final_list = []
        for val in correct_class_order:
            if val not in list(df.columns):
                pass
            else:
                final_list.append(val)

        #convert seconds to datetime
        
        #print("HERE", final_list, list(df.columns), correct_class_order)
        df = df[final_list]
        
        df['SECONDS'] = df['SECONDS'].apply(sec_to_datetime)
        df['SECONDS'] = pd.to_datetime(df['SECONDS'])
        return df


    color_map = {'BRAWLING':'#F2191A', 'RESTRAINING':'#F2191A', 'STRIKING':'#F2191A','THROWING': '#F2191A', 'RUNNING':'#F6AA28', 'CHEMICAL SMOKE':'#30B695', 'CROWD':'#F6AA28',
       'BATON':'#30B695', 'RIOT SHIELD':'#30B695', 'NON-UNIFORMED PERSON':'#4218D9', 'UNIFORMED PERSON':'#4218D9',
       'PERSON ON GROUND':'#F6AA28', 'SPRAY':'#F2191A', 'GUN':'#F2191A', 'PEPPER SPRAY':'#30B695'}

    df = create_df_all_models()

    # calculate bin
    vid_length_sec = len(df) / 2
    if vid_length_sec <= 120:
        bin_size = vid_length_sec * 3
    else:
        bin_size = 240
    
    interval = alt.selection(type='interval', encodings=['x'])

    base = alt.Chart(df).properties(    
        width=1920,
        height=50,
    )

    chart = alt.vconcat(
        data=df,
        spacing=0,)

    for index, y_encoding in enumerate(list(df.columns[1:])):
        orient = 'bottom'
        ticks=False
        labels=False
        bar_color = color_map[y_encoding]

        if index == 0:
            labels = True
            orient = 'top'
            ticks = True
        if index == len(df.columns[1:]) - 1:
            ticks=True
            labels=True

        row = base.mark_bar().encode(
            alt.X('hoursminutesseconds(SECONDS):T', scale=alt.Scale(domain=interval),axis=alt.Axis(tickCount=10,labels=labels, title=None, ticks=ticks, orient=orient),
            ),
            alt.Y(y_encoding, axis=alt.Axis(titleAngle=0, titlePadding=80, labels=False, ticks=False)
            ),
            color=alt.value(bar_color),
            tooltip=[alt.Tooltip(y_encoding, title=f'{y_encoding.title()}'), alt.Tooltip('hoursminutesseconds(SECONDS):T', title='Timestamp')]
            ).properties(
                #width=1920
            )
        chart &= row

    view = base.mark_bar(size=2).encode(
        alt.X('hoursminutesseconds(SECONDS):T', title='Click and drag above to zoom', axis=alt.Axis(tickCount=10, orient='bottom')
        ),
    ).add_selection(
        interval
    )

    both = alt.VConcatChart(
        vconcat=[view, chart],
        spacing=50,
        padding={"left": 10, "top": 20, "right": 50, "bottom": 50},
        
    ).configure_axis(
        gridDash=[1,1],
        grid=True,
    )
    return both

both = generate_visuals(path='/home/bean/code/hrf/prod_evironments/deploy/src/generate_visuals/Augustine-2Marceil-video1_combined.json')

both

alt.VConcatChart(...)